In [1]:
%cd ..

/home/jupyter/sign-language-translation/slt_copy


In [2]:
import sys
sys.path.append('.')
import pickle as pkl
import yaml
from signjoey import data, batch
import gzip
import torch

## let's add pose estimation to  the dataset

In [15]:
with gzip.open('data/PHOENIX2014T/phoenix14t.pami0.dev', "rb") as f:
    loaded_object = pkl.load(f)

with open('data/pose_phoenix2014T_up_hrnet_TxN_wh.pkl', 'rb') as f:
    dict_pose = pkl.load(f)

In [16]:
loaded_object[1]

{'name': 'dev/11August_2010_Wednesday_tagesschau-3',
 'signer': 'Signer08',
 'gloss': 'ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWITTER KOENNEN',
 'text': 'das bedeutet viele wolken und immer wieder zum teil kräftige schauer und gewitter .',
 'sign': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.4179,  ..., 0.0000, 0.0000, 0.0000],
         [0.0043, 0.0000, 2.1061,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.1042, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.4149, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.2398, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])}

In [17]:
loaded_object[1]['sign'].shape

torch.Size([111, 1024])

In [18]:
torch.Tensor(dict_pose[loaded_object[1]['name'].split('/')[1]]).shape

torch.Size([111, 7, 2])

In [27]:
n = torch.Tensor(dict_pose[loaded_object[1]['name'].split('/')[1]]).shape
torch.Tensor(dict_pose[loaded_object[1]['name'].split('/')[1]]).reshape((n[0],14)).

torch.Size([111, 14])

In [28]:
def add_pose(data, pose):
    for i in range(len(data)):
        n = torch.Tensor(dict_pose[data[i]['name'].split('/')[1]]).shape
        data[i]['pose'] = torch.Tensor(dict_pose[data[i]['name'].split('/')[1]]).reshape((n[0], 14))
    return data

def save(data, filename, protocol = 0):
        """Saves a compressed object to disk
        """
        file = gzip.GzipFile(filename, 'wb')
        file.write(pickle.dumps(data, protocol))
        file.close()


**we save the new resulting data set**

In [29]:
for file in ['data/PHOENIX2014T/phoenix14t.pami0.train', 'data/PHOENIX2014T/phoenix14t.pami0.dev', 'data/PHOENIX2014T/phoenix14t.pami0.test']:
    with gzip.open(file, "rb") as f:
        loaded_object = pkl.load(f)
    temp = add_pose(loaded_object, dict_pose)
    outname = 'data/PHOENIX2014&POSE/'+file[18:]
    with open(outname, 'wb') as handle:
        pkl.dump(temp, handle)

**We load it and cheeck compatibility with the modified SignTranslationDataset class to take into account the new poses**

In [30]:
with open("data/PHOENIX2014&POSE/phoenix14t.pami0.test", "rb") as f:
    loaded_object = pkl.load(f)

In [31]:
loaded_object[1]

{'name': 'test/25October_2010_Monday_tagesschau-24',
 'signer': 'Signer01',
 'gloss': 'DONNERSTAG NORDWEST REGEN REGION SONNE WOLKE WECHSELHAFT DANN FREITAG AEHNLICH WETTER',
 'text': 'am donnerstag regen in der nordhälfte in der südhälfte mal sonne mal wolken ähnliches wetter dann auch am freitag .',
 'sign': tensor([[0.0000, 0.0000, 0.5131,  ..., 0.0000, 0.0000, 0.0000],
         [0.0574, 0.0051, 0.5100,  ..., 0.0000, 0.0000, 0.0000],
         [1.3442, 0.1755, 0.1881,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0546, 0.3171, 1.2666,  ..., 0.0000, 0.0000, 0.0000],
         [0.8935, 1.2325, 0.8052,  ..., 0.0061, 0.0000, 0.0000],
         [0.7165, 3.3905, 0.9603,  ..., 0.0000, 0.0000, 0.0000]]),
 'pose': tensor([[[0.4306, 0.2188],
          [0.5972, 0.4583],
          [0.2500, 0.4792],
          ...,
          [0.1806, 0.7188],
          [0.7083, 0.9792],
          [0.1250, 0.9583]],
 
         [[0.4306, 0.2188],
          [0.5972, 0.4479],
          [0.2500, 0.4792],
     

In [3]:
with open(r'configs/sign.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [4]:
train_data, dev_data, test_data, gls_vocab, txt_vocab = data.load_data(config['data'])

In [5]:
len(train_data.examples)

7095

In [6]:
# Pose is effectively on of the fields
train_data.fields

{'sequence': <torchtext.data.field.RawField at 0x7f2ebca6f208>,
 'signer': <torchtext.data.field.RawField at 0x7f2ebca6f320>,
 'sgn': <torchtext.data.field.Field at 0x7f2f114ef6d8>,
 'pose': <torchtext.data.field.Field at 0x7f2ebca6f5c0>,
 'gls': <torchtext.data.field.Field at 0x7f2ebca6f518>,
 'txt': <torchtext.data.field.Field at 0x7f2ebca6f470>}

In [7]:
len(vars(train_data.examples[14])['pose'])

103

In [8]:
len(vars(train_data.examples[14])['sgn'])

103

### **Test**

In [7]:
 
train_iter = data.make_data_iter(
            train_data,
            batch_size=12,
            batch_type='train',
            train=True,
            shuffle=False
        )

In [9]:
for b in train_iter:
#     t = batch
    batch.Batch(
            is_train=True,
            torch_batch=b,
            
            sgn_dim=1024,
            pose_dime = 14
#             use_cuda=self.use_cuda,
#             frame_subsampling_ratio=self.frame_subsampling_ratio,
#             random_frame_subsampling=self.random_frame_subsampling,
#             random_frame_masking_ratio=self.random_frame_masking_ratio,
                    )
    print(t)
    break

TypeError: __init__() missing 1 required positional argument: 'txt_pad_index'

In [15]:
zip([1],[2],[3])

In [58]:
def stack_features(features, something):
#         print(features)
        return torch.stack([torch.stack(ft, dim=0) for ft in features], dim=0)

temp = [vars(train_data.examples[14])['sgn'], vars(train_data.examples[4])['sgn']]

stack_features(temp, '').shape

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 103 and 71 in dimension 1 at /pytorch/aten/src/TH/generic/THTensor.cpp:612

In [54]:
temp = [vars(train_data.examples[14])['pose'], vars(train_data.examples[4])['pose']]
stack_features(temp, '').shape

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 103 and 71 in dimension 1 at /pytorch/aten/src/TH/generic/THTensor.cpp:612

In [55]:
len(vars(train_data.examples[4])['pose'])

71